In [2]:
import os
import glob

import numpy as np
import cv2
import lightgbm as lgbm

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print('Import OK!')

Import OK!


Read images and extract features  
    Use canny algorithm to extract edges from images. Flatten the images into 1D vectors

In [3]:
def loadDataFromDir(**dirNames):
    #Image dimensions
    imH = 80
    imW = 80
    
    #Thresholding for canny edge detection
    minVal = 50
    maxVal = 250


    X = np.array([])
    y = []
    
    for label, dirName in dirNames.items():
        for f in glob.glob(dirName + '/*.png'):
            img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
            img = img[:imH+1, :imW+1] #crop
            edges = cv2.Canny(img, minVal, maxVal) #extract edges
            edges = edges.reshape(np.prod(edges.shape)) #flatten
            
            #append
            if len(X) == 0:
                X = edges
            else:
                X = np.vstack((X, edges))
            
            assert label in ('target', 'nonTarget')
            if(label == 'target'):
                y.append(1)
            elif(label == 'nonTarget'):
                y.append(0)

    return X, y

In [4]:
targetTrainDir = "target_train"
nonTargetTrainDir = "non_target_train"

#Read target data
XTrain, yTrain = loadDataFromDir(target=targetTrainDir, nonTarget=nonTargetTrainDir)
    
print('Feature extraction OK!')

Feature extraction OK!


Train the model  
Use Light Gradient Boosting Machine (use scikit-learn api of ligthgbm)

In [5]:
gbmClassifier = lgbm.LGBMModel(objective='binary', random_state=42)
gbmClassifier.fit(XTrain, yTrain)

print('Training OK!')

Training OK!


Evaluate the model

In [7]:
targetTestDir = "target_dev"
nonTargetTestDir = "non_target_dev"

XTest, yTest = loadDataFromDir(target=targetTestDir, nonTarget=nonTargetTestDir)

predictions = gbmClassifier.predict(XTest)
print(predictions)
predictions = np.round(predictions)
print(predictions)
predictions = predictions.astype(int)

print(predictions)
print(confusion_matrix(yTest, predictions)) #[[TrueNegatives, TruePositives][FalseNegative, FalsePositives]]
print(accuracy_score(yTest, predictions))


[1.54826446e-03 1.49824092e-02 2.11942774e-03 2.85952035e-02
 4.95397927e-02 5.22249095e-02 7.10623212e-03 8.42286084e-02
 5.69187341e-03 4.95308067e-02 1.61312596e-02 7.33128778e-02
 1.70236125e-04 1.87803412e-03 2.47104410e-03 3.23160273e-03
 2.65171993e-02 1.61090304e-02 3.29030473e-03 5.51030835e-03
 8.60686502e-01 4.06835038e-03 3.99315994e-04 3.39666912e-04
 4.92515361e-04 3.24583701e-03 4.91446940e-03 4.53114273e-03
 5.29707325e-04 1.11225379e-03 1.82126049e-04 6.79160603e-03
 3.91174749e-03 2.59850528e-02 5.82785924e-05 2.87439672e-03
 5.56301803e-03 3.38215765e-02 1.28602245e-03 4.69687569e-04
 5.02983387e-02 1.08510380e-01 1.10031292e-03 8.44778517e-04
 1.37598891e-02 3.12684218e-02 2.13709967e-04 8.20949284e-03
 5.13404744e-03 2.19850269e-03 9.02042575e-03 2.10233776e-02
 1.00787296e-01 3.95043713e-02 4.91735277e-04 1.13609263e-02
 1.16521536e-02 2.74966301e-04 2.27506385e-03 8.70196940e-04
 3.20400732e-01 6.44239046e-03 2.81906370e-03 1.11288731e-01
 3.38750681e-02 1.911506